# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [40]:
!python --version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python 3.9.21


In [1]:
# !pip install langchain_community==0.0.9

In [2]:
# #langchain library
# !pip install langchain==0.0.350
# !pip install langchain_community==0.0.4
# # !pip install langchain==0.1.0
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4
# !pip install huggingface_hub==0.20.0


In [3]:
import langchain
print(langchain.__version__)

0.0.350


In [4]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [5]:
from langchain import PromptTemplate

prompt_template = """
    You are AI assistance and you need to answer the user's question according to the context given.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="You are AI assistance and you need to answer the user's question according to the context given.\n    {context}\n    Question: {question}\n    Answer:")

In [6]:
PROMPT.format(
    context = "I'm Nyein Chan Aung, currently working as a product manager at a tech company.",
    question = "Who are you?"
)

"You are AI assistance and you need to answer the user's question according to the context given.\n    I'm Nyein Chan Aung, currently working as a product manager at a tech company.\n    Question: Who are you?\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [7]:
# # connect google drive
# from google.colab import drive
# # set directory with os
# import os
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/_NLP/A6')

In [8]:
# !pip install -U langchain-community

In [9]:
# # update pymupdf
# !pip uninstall pymupdf
# !pip install pymupdf

In [10]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = 'about-nca.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [11]:
# documents

In [12]:
len(documents)

5

In [13]:
documents[1]

Document(page_content='Page 2 of 5 \n \n \nSUMMARY \nDedicated and passionate product expert with 8+ years of experience in various technical industries \n(education, finance, agriculture). My major objective is to contribute to the business by utilizing my \ntechnical and business management skillset. I am eager to improve my experience and knowledge in \n delivering qualified digital products that provide value to clients and users.   \nEDUCATION \nMaster of Data Science and Artificial Intelligence (Msc in DSAI) \n- \nAsian Institute of Technology (2024-Currnet) \n \nBusiness Information Technology (BSc Hons) \n• University of Greenwich (2011 - 2014) \n \nBachelor of Laws (LL. B) \n • Yangon University of Distance Education, Yangon, Myanmar (2007 - 2010) \nWORK EXPERIENCE \nProduct Manager \nVillage Link Co. Ltd. \nNov 2019 to Present \n \nJob Responsibility \n• Assist the CEO in preparing product roadmaps for business development efforts. \n• Prepare Product Requirement Documents (P

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [15]:
doc[1]

Document(page_content='allowed me to migrate the hundred thousand user-generated datasets from the legacy system \nto a completely new platform in 2021. \n \nI started my career in the tech industry as a developer while I was studying Business \nInformation Technology. After that, I moved forward into product development and \nmanagement when I found my enthusiasm for the art of product development lifecycle \napplying Agile Methodology. However, I am still actively sharing the technical workload by \nreviewing the source code for quality assurance and taking responsibility for the server \ndeployment using different cloud service providers such as AWS, Microsoft Azure, and Digital \nOcean.', metadata={'source': 'about-nca.pdf', 'file_path': 'about-nca.pdf', 'page': 0, 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creationDate': "D:20250316085053Z00'00'

In [16]:
len(doc)

17

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [17]:
# !pip install --upgrade huggingface_hub==0.18.0

In [18]:
import huggingface_hub
print(huggingface_hub.__version__)

0.20.0


In [19]:
print(huggingface_hub.__version__)

0.20.0


In [20]:
import torch
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import OpenAIEmbeddings


model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/opt/anaconda3/envs/oldrag/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
/opt/anaconda3/envs/oldrag/lib/python3.9/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/opt/anaconda3/envs/oldrag/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/envs/oldrag/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [21]:
#locate vectorstore
vector_path = 'vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [22]:
# !pip install faiss-cpu

In [23]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [24]:
#calling vector from local
vector_path = 'vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [25]:
#ready to use
retriever = vectordb.as_retriever()

In [26]:
retriever.get_relevant_documents("What is your education background?", top_k = 1)

[Document(page_content='Page 2 of 5 \n \n \nSUMMARY \nDedicated and passionate product expert with 8+ years of experience in various technical industries \n(education, finance, agriculture). My major objective is to contribute to the business by utilizing my \ntechnical and business management skillset. I am eager to improve my experience and knowledge in \n delivering qualified digital products that provide value to clients and users.   \nEDUCATION \nMaster of Data Science and Artificial Intelligence (Msc in DSAI) \n- \nAsian Institute of Technology (2024-Currnet) \n \nBusiness Information Technology (BSc Hons) \n• University of Greenwich (2011 - 2014) \n \nBachelor of Laws (LL. B)', metadata={'source': 'about-nca.pdf', 'file_path': 'about-nca.pdf', 'page': 1, 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creationDate': "D:20250316085053Z00'00'", 'modD

In [27]:
retriever.get_relevant_documents("What is your working experience?", top_k = 1)

[Document(page_content='Page 2 of 5 \n \n \nSUMMARY \nDedicated and passionate product expert with 8+ years of experience in various technical industries \n(education, finance, agriculture). My major objective is to contribute to the business by utilizing my \ntechnical and business management skillset. I am eager to improve my experience and knowledge in \n delivering qualified digital products that provide value to clients and users.   \nEDUCATION \nMaster of Data Science and Artificial Intelligence (Msc in DSAI) \n- \nAsian Institute of Technology (2024-Currnet) \n \nBusiness Information Technology (BSc Hons) \n• University of Greenwich (2011 - 2014) \n \nBachelor of Laws (LL. B)', metadata={'source': 'about-nca.pdf', 'file_path': 'about-nca.pdf', 'page': 1, 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creationDate': "D:20250316085053Z00'00'", 'modD

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [28]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [29]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [30]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [31]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [32]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [33]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [34]:
# !pip install protobuf

In [35]:
# from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from transformers import BitsAndBytesConfig
# from langchain import HuggingFacePipeline
# import torch

# model_id = 'fastchat-t5-3b-v1.0/'

# tokenizer = AutoTokenizer.from_pretrained(
#     model_id)

# tokenizer.pad_token_id = tokenizer.eos_token_id

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = "nf4",
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = True
# )

# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_id,
#     quantization_config = bitsandbyte_config, #caution Nvidia
#     device_map = 'auto',
#     load_in_8bit = True
# )

# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens = 256,
#     model_kwargs = {
#         "temperature" : 0,
#         "repetition_penalty": 1.5
#     }
# )

# llm = HuggingFacePipeline(pipeline = pipe)

In [36]:
# !pip install safetensors

In [37]:
# from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from langchain import HuggingFacePipeline
# import torch

# # Use a smaller model (e.g., t5-small)
# model_id = 't5-small'

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token_id = tokenizer.eos_token_id

# # Load the model
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_id,
#     device_map='auto',  # Automatically map to CPU or GPU
#     torch_dtype=torch.float32  # Use FP32 for CPU
# )

# # Create a text generation pipeline
# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=256,
#     model_kwargs={
#         "temperature": 0,
#         "repetition_penalty": 1.5
#     }
# )

# # Wrap the pipeline in LangChain's HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

In [38]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Load the TinyLlama model and tokenizer
# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

In [39]:
# from transformers import pipeline

# # Create a text generation pipeline with greedy decoding
# pipe = pipeline(
#     task="text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=100,
#     do_sample=False,  # Disable sampling (greedy decoding)
#     repetition_penalty=1.5  # Penalize repeated tokens
# )

# # Generate text
# input_text = "What is your name?"
# output = pipe(input_text)
# print(output[0]['generated_text'])

In [45]:
from huggingface_hub import snapshot_download

# Download the model files
model_path = snapshot_download(repo_id='lmsys/fastchat-t5-3b-v1.0')
print(f"Model downloaded to: {model_path}")

generation_config.json: 100%|██████████| 142/142 [00:00<00:00, 53.9kB/s]
pytorch_model.bin.index.json: 100%|██████████| 50.8k/50.8k [00:00<00:00, 13.1MB/s]
README.md: 100%|██████████| 1.98k/1.98k [00:00<00:00, 1.08MB/s]
.gitattributes: 100%|██████████| 1.48k/1.48k [00:00<00:00, 983kB/s]
Fetching 10 files: 100%|██████████| 10/10 [00:01<00:00,  5.94it/s]

Model downloaded to: /Users/nyeinchanaung/.cache/huggingface/hub/models--lmsys--fastchat-t5-3b-v1.0/snapshots/0b1da230a891854102d749b93f7ddf1f18a81024


In [46]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain import HuggingFacePipeline
import torch

# Create an offload folder
os.makedirs("./offload", exist_ok=True)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    device_map='auto',
    offload_folder="./offload",  # Specify the offload folder
    torch_dtype=torch.float32  # Use FP32 for CPU
)

# Create a text generation pipeline
pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    model_kwargs={
        "temperature": 0,
        "repetition_penalty": 1.5
    }
)

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [49]:
# from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from transformers import BitsAndBytesConfig
# from langchain import HuggingFacePipeline
# import torch

# model_id = 'fastchat-t5-3b-v1.0/'

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token_id = tokenizer.eos_token_id

# # Remove BitsAndBytesConfig (needed only for GPU quantization)
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_id,
#     device_map=None,  # Force CPU
#     load_in_8bit=False  # Disable 8-bit quantization
# )

# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=256,
#     model_kwargs={
#         "temperature": 0,
#         "repetition_penalty": 1.5
#     }
# )

# llm = HuggingFacePipeline(pipeline=pipe)

In [172]:
from langchain import HuggingFacePipeline

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

# Use the LLM in LangChain
response = llm("What is your name?")
print(response)


I'm Sarah. How can I help you today, Ms. Johnson?"


In [156]:
# # Define the retrieval chain

# llm = HuggingFacePipeline.from_model_id("TinyLlama/TinyLlama-1.1B-Chat-v1.0", task="text-generation", device=-1)


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [50]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [51]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [52]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [53]:
# query = 'Comparing both of them'
# chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

# question_generator({'chat_history' : chat_history, "question" : query})

`combine_docs_chain`

In [54]:

doc_chain = PromptTemplate(
    input_variables=["context", "question"],
    template="Answer the question based on the context below. If the context does not contain the answer, say 'I don't know.'\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:",
)

doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are AI assistance and you need to answer the user's question according to the context given.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x5e572dbe0>)), document_variable_name='context')

In [55]:
query = "What is your name?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are AI assistance and you need to answer the user's question according to the context given.
    Page 1 of 5 
 
Name: Nyein Chan Aung 
Position: Digital Product & Project Professional 
Address: Yangon, Myanmar 
Phone: 091212121 
 
My name is Nyein Chan Aung. I was born in 7th Aug, 1990.  I’m currently working as a Product 
Manager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for 
product management, UI/UX research, user analysis, QA testing, and the product launch of our 
company’s many products. I am leading the development of a delivery social agricultural 
platform with registered farmers. My ability to understand the system and database design 
allowed me to migrate the hundred thousand user-generated datasets from the legacy system

allowed me to migrate the hundred thousand user-generated datasets from the legacy system 
to a completely

{'input_documents': [Document(page_content='Page 1 of 5 \n \nName: Nyein Chan Aung \nPosition: Digital Product & Project Professional \nAddress: Yangon, Myanmar \nPhone: 091212121 \n \nMy name is Nyein Chan Aung. I was born in 7th Aug, 1990.  I’m currently working as a Product \nManager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for \nproduct management, UI/UX research, user analysis, QA testing, and the product launch of our \ncompany’s many products. I am leading the development of a delivery social agricultural \nplatform with registered farmers. My ability to understand the system and database design \nallowed me to migrate the hundred thousand user-generated datasets from the legacy system', metadata={'source': 'about-nca.pdf', 'file_path': 'about-nca.pdf', 'page': 0, 'total_pages': 5, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext'

In [56]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="You are AI assistance and you need to answer the user's question according to the context given.\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x5e572dbe0>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'), llm=Huggin

## 5. Chatbot

In [57]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are AI assistance and you need to answer the user's question according to the context given.
    Page 1 of 5 
 
Name: Nyein Chan Aung 
Position: Digital Product & Project Professional 
Address: Yangon, Myanmar 
Phone: 091212121 
 
My name is Nyein Chan Aung. I was born in 7th Aug, 1990.  I’m currently working as a Product 
Manager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for 
product management, UI/UX research, user analysis, QA testing, and the product launch of our 
company’s many products. I am leading the development of a delivery social agricultural 
platform with registered farmers. My ability to understand the system and database design 
allowed me to migrate the hundred thousand user-generated datasets from the legacy system

allowed me to migrate the hundred thousand user-gen

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "<pad>  I  am  an  AI  assistance  and  I  don't  have  a  name  or  position.  I  am  a  text-based  AI  language  model  that  can  provide  information  based  on  the  context  provided.  I  can  answer  the  user's  question  based  on  the  information  provided  in  the  context.\n",
 'source_documents': [Document(page_content='Page 1 of 5 \n \nName: Nyein Chan Aung \nPosition: Digital Product & Project Professional \nAddress: Yangon, Myanmar \nPhone: 091212121 \n \nMy name is Nyein Chan Aung. I was born in 7th Aug, 1990.  I’m currently working as a Product \nManager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for \nproduct management, UI/UX research, user analysis, QA testing, and the product launch of our \ncompany’s many products. I am leading the development of a delivery social agricultural \nplatform with registered farmers. My ability to understand the system and d

In [58]:
prompt_question = "WWhat is your highest level of education?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I  am  an  AI  assistance  and  I  don't  have  a  name  or  position.  I  am  a  text-based  AI  language  model  that  can  provide  information  based  on  the  context  provided.  I  can  answer  the  user's  question  based  on  the  information  provided  in  the  context.\n")]
Follow Up Input: WWhat is your highest level of education?
Standalone question:



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are AI assistance and you need to answer the user's question according to the context given.
    Page 2 of 5 
 
 
SUMMARY 
Dedicated and passionate product expert with 8+ years of experience in various technical industries 
(education, finance, agriculture). My major objective is to contribute to the business by utilizing my 
technical and business management skillset. I am eager to improve my experience and knowledge in 
 delivering qualified digital products that provide value to clients and users.   
EDUCATION 
Master of Data Science and Artificial Intelligence (Msc in DSAI) 
- 
Asian Institute of Technology (2024-Currnet) 
 
Business Information Technology (BSc Hons) 
• University of Greenwich (2011 - 2014) 
 
Bachelor of Laws (LL. B)

• University of Greenwich (2011 - 2014) 
 
Bachelor of Laws (LL. B) 
 • Yangon University of Distance Education, Yangon, 

{'question': 'WWhat is your highest level of education?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad>  I  am  an  AI  assistance  and  I  don't  have  a  name  or  position.  I  am  a  text-based  AI  language  model  that  can  provide  information  based  on  the  context  provided.  I  can  answer  the  user's  question  based  on  the  information  provided  in  the  context.\n")],
 'answer': '<pad>  My  highest  level  of  education  is  a  Master  of  Data  Science  and  Artificial  Intelligence  (Msc  in  DSAI)  from  Asian  Institute  of  Technology.\n',
 'source_documents': [Document(page_content='Page 2 of 5 \n \n \nSUMMARY \nDedicated and passionate product expert with 8+ years of experience in various technical industries \n(education, finance, agriculture). My major objective is to contribute to the business by utilizing my \ntechnical and business management skillset. I am eager to improve my experience and knowledge in \n

In [59]:
prompt_question = "What major or field of study did you pursue during your education?"
answer = chain({"question":prompt_question})
answer 



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad>  I  am  an  AI  assistance  and  I  don't  have  a  name  or  position.  I  am  a  text-based  AI  language  model  that  can  provide  information  based  on  the  context  provided.  I  can  answer  the  user's  question  based  on  the  information  provided  in  the  context.\n"), HumanMessage(content='WWhat is your highest level of education?'), AIMessage(content='<pad>  My  highest  level  of  education  is  a  Master  of  Data  Science  and  Artificial  Intelligence  (Msc  in  DSAI)  from  Asian  Institute  of  Technology.\n')]
Follow Up Input: What major or field of study did you pursue during your education?
Standalone question:

> 

{'question': 'What major or field of study did you pursue during your education?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad>  I  am  an  AI  assistance  and  I  don't  have  a  name  or  position.  I  am  a  text-based  AI  language  model  that  can  provide  information  based  on  the  context  provided.  I  can  answer  the  user's  question  based  on  the  information  provided  in  the  context.\n"),
  HumanMessage(content='WWhat is your highest level of education?'),
  AIMessage(content='<pad>  My  highest  level  of  education  is  a  Master  of  Data  Science  and  Artificial  Intelligence  (Msc  in  DSAI)  from  Asian  Institute  of  Technology.\n')],
 'answer': '<pad>  My  major  or  field  of  study  during  my  education  was  Business  Information  Technology  (BSc  Hons).\n',
 'source_documents': [Document(page_content='Page 2 of 5 \n \n \nSUMMARY \nDedicated and passionate product expert with 8+ years of experience in

In [61]:
# prompt_question = "How many years of work experience do you have?"
# answer = chain({"question":prompt_question})
# answer

In [62]:
# prompt_question = "How old are you?"
# answer = chain({"question":prompt_question})
# answer